In [1]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from models import GCN
import importlib
import utils
importlib.reload(utils)
from utils import *

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)


In [2]:
# get paths to available grahs metadata
N_sample = 200000
adj_mat, features_mat, train_labels_mat, test_labels_mat, val_labels_mat, train_mask_mat, test_mask_mat, val_mask_mat = load_data("txt_graph2216_21012020", N_sample)

In [3]:
print(adj_mat.shape)
print(features_mat.shape)
print(train_labels_mat.shape)
print(test_labels_mat.shape)
print(val_labels_mat.shape)
print(train_mask_mat.shape)
print(test_mask_mat.shape)
print(val_mask_mat.shape)

print("number of masked elements")
print("train: " + str(len(np.where(train_mask_mat[:,1]==1)[0])))
print("test: " + str(len(np.where(test_mask_mat[:,1]==1)[0])))
print("val: " + str(len(np.where(val_mask_mat[:,1]==1)[0])))

print("\n positions of ones in masked labels")
print(np.where(train_labels_mat[np.array(train_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(test_labels_mat[np.array(test_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(val_labels_mat[np.array(val_mask_mat[:,1],dtype=bool)]==1)[1])

print("\n ones outside the mask")
print(np.where(train_labels_mat[np.logical_not(np.array(train_mask_mat[:,1],dtype=bool))]==1))
print(np.where(test_labels_mat[np.logical_not(np.array(test_mask_mat[:,1],dtype=bool))]==1))
print(np.where(val_labels_mat[np.logical_not(np.array(val_mask_mat[:,1],dtype=bool))]==1))

#sum_global_nodes = np.sum(adj_mat.toarray()[:,np.array(train_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(test_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(val_mask_mat[:,1],dtype=bool)], axis = 1)
#print(adj_mat.shape[1] - sum(sum_global_nodes))

(465542, 465542)
(465542, 100)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
number of masked elements
train: 779
test: 668
val: 778

 positions of ones in masked labels
[2 1 0 4 0 4 4 1 3 0 3 3 0 4 2 0 0 1 4 1 1 3 0 2 0 1 1 0 4 0 4 3 3 2 2 0 2
 2 4 4 1 3 2 2 0 4 4 4 3 1 2 0 3 1 2 0 1 3 4 0 3 3 3 4 0 1 4 1 3 4 3 1 1 1
 3 0 0 4 3 2 1 0 0 0 2 0 4 3 0 0 4 4 3 2 3 3 1 4 3 4 1 0 3 2 4 3 4 4 0 1 4
 3 1 0 1 3 3 4 4 3 3 3 4 3 2 3 1 2 3 1 3 0 2 0 3 2 4 2 0 2 0 4 2 0 2 1 0 2
 0 0 3 0 1 2 3 3 3 1 4 3 2 4 3 3 0 4 3 1 3 1 3 0 4 2 4 2 0 3 2 3 2 2 2 0 2
 2 4 0 2 3 0 2 0 0 0 2 0 0 2 2 2 3 2 0 3 1 4 3 0 0 4 1 2 2 4 0 2 3 0 4 2 3
 0 0 2 1 0 0 3 3 3 1 2 4 0 3 0 3 3 1 4 3 2 4 2 3 4 0 3 4 0 3 3 4 0 0 0 3 3
 0 2 1 4 4 3 0 0 4 4 1 1 4 3 0 3 4 3 2 3 2 1 4 3 4 0 3 0 4 4 0 4 1 2 2 3 3
 3 0 2 4 3 3 2 3 4 2 1 0 2 4 4 1 4 3 4 3 1 3 3 1 3 2 0 4 1 0 2 3 0 4 4 4 1
 3 3 4 1 0 4 2 3 3 0 0 1 2 3 1 0 4 0 2 3 1 1 0 2 3 3 1 0 4 3 3 2 4 0 0 1 1
 0 0 0 2 0 0 3 3 4 3 1 1 0 4 2 1 0 0 3 1 4 1 3 0 3 1 2

In [15]:
#delete all flags before declaration new one
del_all_flags(tf.flags.FLAGS)

# Settings
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('mode', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('port', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('f', '', 'kernel') # for jupyter notebook

flags.DEFINE_float('learning_rate', 0.03, 'Initial learning rate.') #0.01
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.') #200
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.') #16
flags.DEFINE_float('dropout', 0.6, 'Dropout rate (1 - keep probability).') #0.5
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 20, 'Tolerance for early stopping (# of epochs).') #10

# Some preprocessing
features = preprocess_features(features_mat)
support = [preprocess_adj(adj_mat)]
num_supports = 1
model_func = GCN 

/home/robertb/python/GCN_text_classif/gcn/utils.py:57: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [5]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, train_labels_mat.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# Initialize session
sess = tf.Session()
# Init variables
sess.run(tf.global_variables_initializer())

In [ ]:
# Train model
cost_val = []

for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, train_labels_mat, train_mask_mat[:,1], placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, val_labels_mat, val_mask_mat[:,1], placeholders, sess, model)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

In [12]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, test_labels_mat, test_mask_mat[:,1], placeholders, sess, model)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

Test set results: cost= 1.61192 accuracy= 0.27121 time= 6.48976
[39.417572, 37.68326, 36.662716, 35.950226, 35.040947, 34.0105, 32.79931, 31.685692, 30.612103, 29.522766, 28.421473, 27.267614, 26.04314, 24.885725, 23.816912, 22.911482, 23.14679, 23.502064, 23.898617, 24.375204, 24.817476, 25.239853, 25.526617, 25.858856, 26.139387, 26.424393, 26.627184, 26.812918, 27.091627, 27.36971, 27.593872, 27.979736, 28.283022, 28.618864, 29.029453, 29.448526, 29.876125, 30.279856, 30.6507, 30.958248, 31.245161, 31.501953, 31.7274, 31.950003, 32.24564, 32.449863, 32.61364, 32.748634, 32.787884, 32.878876, 32.916332, 32.883392, 32.85091, 32.859634, 32.95164, 32.96809, 32.98352, 33.06282, 33.126617, 33.15312, 33.240543, 33.33949, 33.372524, 33.442852, 33.45361, 33.35721, 33.25648, 33.16203, 33.048542, 32.84707, 32.676517, 32.422215, 32.205143, 31.94729, 31.657776, 31.421373, 31.261932, 31.153067, 31.088106, 31.04863, 30.961594, 30.857067, 30.753826, 30.67305, 30.616137, 30.582897, 30.563549, 30.482